# README
This document does not correctly solve the problem, since the sum of values in grids is hard-coded. 

Instead, see `main.py`, where all valid grid sums are generated, and `KnightsMoveSolver.py` is used to place the values according to the gridsums.

### Trying to work out valid grid sums
The matrix is filled with $1$, $2$, $\dots$, $N$, for some unkown $N$, summing to $N(N-1)/2$. 

* Since 33 is already filled in, $N \geq 33$. 
* Since the matrix is $10 \times 10$, then $N \leq 100$
* There are 17 grids, so $N(N-1)$ should be divisible by 17. 
* There is a 2-grid near the bottom left corner (yellow, grid 14 in the code).
    * If both values are filled, then the sum must be odd due to parity arguments. 
        * *don't think this is useful*
    * If both values are filled, then the maximum grid-sum is at most 100+99 = 199.
Valid combinations

| $N$ | $N-1$ | $N (N-1)/2$ | grid sum |
|-----|-------|-------------|----------|
| 34  |  33   |  561        | 33       |
| 35  |  34   |  595        | 35       |
| 51  |  50   |  1275       | 75       |
| ... |


#### Load CP-SAT and test values

In [31]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()

from tests import ex_vals_list, ex_vals, ex_grid
vals_list = ex_vals_list
grid = ex_grid

N, _ = grid.shape


#### Problem setup
**Grid sum**
* The matrix is filled with $1$, $2$, $\dots$ up to some unknown $N$.
* The sum of all values is $\sum_{i=1}^{N} i = \frac{N (N-1)}{2}$.

In [ ]:
ngrids = grid.max()

gridsum_target = 15 if N == 5 else 75 # hard-coded

nrange = [n for n in range(N**2)] # (0 to N^2-1)
vrange = [v for v in range(N**2+1)] # (0 to N^2)
Grange = [g+1 for g in range(ngrids)] # grid index

#### Utility for Knight's moves
Setup matrix $e_{n_1, n_2}$ to indicate whether cells $n_1$ and $n_2$ are within a knight's move with each other.

In [32]:
def n2ij(n):
    i = n // N
    j = n - i*N
    return (i,j)
def ij2n(i,j):
    return i*N + j

def isKnightsMove(n1, n2):
    '''
    True if n1 and n2 are knights move away
    '''
    i1, j1 = n2ij(n1)
    i2, j2 = n2ij(n2)
    di = abs(i2-i1)
    dj = abs(j2-j1)
    if (di == 2 and dj == 1) or (di == 1 and dj == 2):
        return 1
    else:
        return 0

e = [[isKnightsMove(n1,n2) if n1 != n2 else 0 for n1 in nrange] for n2 in nrange]

#### Setup variables:
* $x_{n,v} = 1$ $\Leftrightarrow$ cell $n$ (row-major 0-based indexing) contains value $v$.
    * Note that $v=0$ is used to indicate that no value is in a cell.

In [33]:
## Define and save variables
x = {}
for n in nrange:
    for v in vrange:
        x[n,v] = model.NewBoolVar(f'x[{n},{v}]')
# f = {}
# for n1, n2 in product(nrange, nrange):
#     f[n1,n2] = model.NewBoolVar(f'f[{n1},{n2}]')


#### Uniqueness constraints
* Each cell $n$ should only be associated with one value $v$.
* Each value $v>0$ should only appear at most once.

In [34]:
## UNIQUENESS constraints
# each (n) has exactly one (v) (including 0)
for n in nrange:
    model.AddExactlyOne(x[n,v] for v in vrange)
# each (v>0) allocated at exactly one (n)
for v in vrange:
    if v > 0:
        model.AddAtMostOne(x[n,v] for n in nrange)


### Knight's move constraints
* Consider a tuple $(n,v)$. 
    * If $x_{n,v} = 1$, then $\sum_{n'} x_{n',v+1}$ should be $0$ for all $n'$ **not** within Knight's move of $n$ (i.e. no invalid step taken).
    * If $x_{n,v} = 0$, then $\sum_{n'} x_{n',v+1}$ can be either $0$ or $1$, i.e. found or not found, doesn't really matter $\Leftrightarrow$ no real constraint applied on where $v+1$ occurs.

    The following equation satisfies the above constraints:
    * $\sum_{n'} x_{n', v+1} \leq 1-x_{n,v}$ for all $n$ and all $v \in [1, N^2-1]$, where $n'$ is **not** within Knight's move of $n$.

* If $v$ exists, then so should $v-1$.
    * If $x_{n,v} = 1$, then $\sum_n x_{n, v-1} = 1$ for all $v-1 \geq 1$.
    * If $x_{n,v} = 0$, then $\sum_n x_{n, v-1}$ can be either $0$ or $1$.

    The following equation satisfies the above constraints:
    * $\sum_{n} x_{n, v-1} \geq x_{n,v}$ for all $n$ and all $v \in [2, N^2]$.

In [35]:
## Knight's move constraint
for n in nrange:
    for v in vrange:
        if v >= 1 and v <= N**2-1:
            nn = [i for i in nrange if i != n and not e[i][n]]
            model.Add(sum(x[i,v+1] for i in nn) <= 1 - x[n,v])
        if v >= 2 and v <= N**2:
            model.Add(sum(x[i, v-1] for i in nrange) >= x[n,v])
# each (n1) steps to at most one (n2)
# for n1 in nrange:
#     model.AddAtMostOne(f[n1,n2] for n2 in nrange)
#     model.AddAtMostOne(f[n2,n1] for n2 in nrange) # likewise from

# # feasible steps
# model.Add(f[n1,n2] <= e[n1,n2] for n1, n2 in product(nrange, nrange))

# v+1, v are bound by knights moves
# 


#### Instance constraints
* Cell $n'$ contains value $v'$:
    * $x_{n', v'} = 1$
* Sum of values in each sub-grid $g'$ sums to the correct value:
    * $\sum_{n' \in g} \sum_v v x_{n',v} = grid\_sum$ 

In [36]:
# prescribed values
for (i,j), val in vals_list:
    model.Add(x[ij2n(i,j),val] == 1)
# correct grid sum
for g in Grange:
    subgrid_sum = sum(v*x[n,v] for n in nrange for v in nrange if grid[n2ij(n)] == g)
    model.Add(subgrid_sum == gridsum_target)

# SOLVE
status = solver.Solve(model)